In [8]:
# ⬇️ 1.  AUTOMATYCZNA INSTALACJA POTRZEBNYCH PAKIETÓW
!pip install --quiet pandas openpyxl

# ⬇️ 2.  CAŁY SKRYPT CZYSZCZENIA + OBSŁUGA WGRYWANIA I POBIERANIA
import re, pandas as pd, os, sys
from google.colab import files

# --- Funkcja pomocnicza: wyciąganie wersji ---
def extract_versions(field: str, prod=False):
    if not isinstance(field, str):
        return ''
    key = r'AMProductVersion' if prod else r'AMEngineVersion'
    m = re.search(key + r'=([^;{}]+)', field)
    return m.group(1) if m else field.strip()

# --- Parser pojedynczej linii ---
_key_re = re.compile(r'(aid|hostname|gpostate|engine|product)\s*[:=]\s*', re.I)

def parse_line(line: str):
    """Zwraca dict z AID, Hostname, GPO state, engine, product."""
    matches = list(_key_re.finditer(line))
    row = {k: '' for k in ('AID', 'Hostname', 'GPO state', 'engine', 'product')}
    for i, m in enumerate(matches):
        key = m.group(1).lower()
        start_val = m.end()
        end_val = matches[i + 1].start() if i + 1 < len(matches) else len(line)
        val = line[start_val:end_val].strip().rstrip(' ,')

        if key == 'aid':
            # jeżeli AID zaczyna się klamrą {id1,id2}, weź pierwszy
            if val.startswith('{'):
                val = val.strip('{}').split(',')[0].strip()
            row['AID'] = val
        elif key == 'hostname':
            row['Hostname'] = val
        elif key == 'gpostate':
            row['GPO state'] = val.replace(' ', '')  # usuwamy spacje
        elif key == 'engine':
            row['engine'] = val         # pełny obiekt @{AMEngineVersion=...}
            # row['engine'] = extract_versions(val, prod=False)   # tylko numer
        elif key == 'product':
            row['product'] = val        # pełny obiekt @{AMProductVersion=...}
            # row['product'] = extract_versions(val, prod=True)   # tylko numer
    return row

# --- 3.  Wgrywanie surowego CSV ---
print("Wybierz swój plik CSV (np. CorporateServerAssessment.csv)…")
uploaded = files.upload()
if not uploaded:
    sys.exit("❌ Nie wczytano żadnego pliku.")

raw_file = next(iter(uploaded))
print(f"✅ Załadowano: {raw_file}")

# --- 4.  Przetwarzanie ---
records = []
with open(raw_file, encoding='utf-8', errors='ignore') as fh:
    for ln in fh:
        r = parse_line(ln)
        if r['AID']:            # ignoruj puste wiersze
            records.append(r)

df = pd.DataFrame(records, columns=['AID', 'Hostname', 'GPO state', 'engine', 'product'])

# --- 5.  Podgląd i zapis do XLSX ---
print("\nPodgląd pierwszych 5 wierszy po czyszczeniu:")
display(df.head())

out_file = os.path.splitext(raw_file)[0] + '_clean.xlsx'
df.to_excel(out_file, index=False, engine='openpyxl')
print(f"\n✅ Zapisano wynik do: {out_file}")

# --- 6.  Pobranie na komputer ---
files.download(out_file)
print("⬇️  Kliknij powyżej, aby pobrać plik na dysk.")


Wybierz swój plik CSV (np. CorporateServerAssessment.csv)…


Saving CorporateServerAssessment.csv to CorporateServerAssessment (2).csv
✅ Załadowano: CorporateServerAssessment (2).csv

Podgląd pierwszych 5 wierszy po czyszczeniu:


,AID,Hostname,GPO state,engine,product
0,2b0bd430d9e144b58d156c67bdbac670,AVS-TEST-VM,,@{AMEngineVersion=1.1.25050.6; PSComputerName=...,@{AMProductVersion=4.18.25050.5; PSComputerNam...
1,08af51e9f5b84fb5910c7cec7e72eb69,CHEYSPVSWFS002,DisableMDEGPOactive,@{AMEngineVersion=1.1.25050.6; PSComputerName=...,@{AMProductVersion=4.18.25050.5; PSComputerNam...
2,4ce04cc37979416993125ceecf2d3caf,cloudOps,,,
3,5387c6c32a8843e4a14e311298eefa28,CloudOpsTest,,,
4,973dd1a9e9364881b9d9143f1650ca54,DCAPWINTE001,DisableMDEGPOactive,@{AMEngineVersion=1.1.25050.6; PSComputerName=...,@{AMProductVersion=4.18.25050.5; PSComputerNam...



✅ Zapisano wynik do: CorporateServerAssessment (2)_clean.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⬇️  Kliknij powyżej, aby pobrać plik na dysk.
